In [27]:
import pandas as pd
import numpy as np

In [28]:
# Chargement du dataset Kaggle
df_kaggle = pd.read_csv("../../Data/2025/atp_tennis_2025.csv")

# Conversion de la colonne de date au bon format
df_kaggle["Date"] = pd.to_datetime(df_kaggle["Date"], errors='coerce')

# Filtrage sur l'année 2025 uniquement
df_2025 = df_kaggle[df_kaggle["Date"].dt.year == 2025].copy()

print(f"{len(df_2025)} matches found for 2025")
df_2025.head()

958 matches found for 2025


,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
64181,Brisbane International,2025-01-01,ATP250,Outdoor,Hard,2nd Round,3,Nishioka Y.,Lehecka J.,Lehecka J.,69,28,776,1660,3.20,1.36,2-6 7-6 4-6
64182,Brisbane International,2025-01-01,ATP250,Outdoor,Hard,2nd Round,3,Jarry N.,Bonzi B.,Jarry N.,35,75,1370,730,2.63,1.50,6-2 6-3
64183,Brisbane International,2025-01-01,ATP250,Outdoor,Hard,2nd Round,3,Vukic A.,Dimitrov G.,Dimitrov G.,68,10,778,3350,4.50,1.20,2-6 6-7
64184,Brisbane International,2025-01-01,ATP250,Outdoor,Hard,2nd Round,3,Thompson J.,Michelsen A.,Thompson J.,26,41,1745,1245,1.67,2.20,7-5 6-3
64185,Brisbane International,2025-01-02,ATP250,Outdoor,Hard,2nd Round,3,Lajovic D.,Mensik J.,Mensik J.,81,48,710,1136,4.00,1.25,3-6 2-6


In [29]:
# Dataset Jeff Sackmann pour structure de référence
df_jeff = pd.read_csv("../../Data/atp/atp_matches_2024.csv")
df_jeff.head()


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2024-0339,Brisbane,Hard,32,A,20240101,300,105777,2.0,NaN,...,58.0,44.0,16.0,11.0,8.0,9.0,14.0,2570.0,8.0,3660.0
1,2024-0339,Brisbane,Hard,32,A,20240101,299,208029,1.0,NaN,...,35.0,31.0,10.0,11.0,5.0,7.0,8.0,3660.0,39.0,1122.0
2,2024-0339,Brisbane,Hard,32,A,20240101,298,105777,2.0,NaN,...,39.0,24.0,14.0,10.0,5.0,7.0,14.0,2570.0,55.0,902.0
3,2024-0339,Brisbane,Hard,32,A,20240101,297,208029,1.0,NaN,...,51.0,31.0,16.0,10.0,3.0,5.0,8.0,3660.0,116.0,573.0
4,2024-0339,Brisbane,Hard,32,A,20240101,296,126128,NaN,NaN,...,37.0,27.0,16.0,10.0,5.0,8.0,39.0,1122.0,44.0,1021.0


In [30]:
# Colonnes à harmoniser
cols_jeff = set(df_jeff.columns)
cols_kaggle = set(df_2025.columns)

missing_in_kaggle = cols_jeff - cols_kaggle
extra_in_kaggle = cols_kaggle - cols_jeff

print("Colonnes manquantes dans Kaggle :", missing_in_kaggle)
print("Colonnes supplémentaires dans Kaggle :", extra_in_kaggle)


print("Colonnes dans jeff :", cols_jeff)
print("Colonnes dans Kaggle :", cols_kaggle)

Colonnes manquantes dans Kaggle : {'loser_name', 'winner_name', 'round', 'winner_ht', 'loser_age', 'winner_ioc', 'winner_rank', 'winner_entry', 'loser_ht', 'loser_rank_points', 'winner_age', 'surface', 'winner_id', 'l_1stWon', 'w_1stIn', 'w_df', 'loser_ioc', 'w_1stWon', 'winner_hand', 'loser_seed', 'w_SvGms', 'score', 'l_SvGms', 'best_of', 'loser_entry', 'winner_rank_points', 'tourney_id', 'l_bpSaved', 'loser_id', 'w_bpFaced', 'loser_rank', 'tourney_level', 'l_1stIn', 'tourney_name', 'winner_seed', 'w_ace', 'l_ace', 'minutes', 'tourney_date', 'match_num', 'draw_size', 'loser_hand', 'w_svpt', 'w_bpSaved', 'l_bpFaced', 'l_df', 'l_svpt', 'l_2ndWon', 'w_2ndWon'}
Colonnes supplémentaires dans Kaggle : {'Player_1', 'Odd_1', 'Round', 'Rank_2', 'Pts_1', 'Score', 'Court', 'Date', 'Winner', 'Rank_1', 'Odd_2', 'Series', 'Surface', 'Pts_2', 'Best of', 'Player_2', 'Tournament'}
Colonnes dans jeff : {'loser_name', 'winner_name', 'round', 'winner_ht', 'loser_age', 'winner_ioc', 'winner_rank', 'winner

In [42]:
# 1. Dictionnaire de renommage (Kaggle → Jeff)
rename_cols = {
    "Player_1": "winner_name",
    "Player_2": "loser_name",
    "Rank_1": "winner_rank",
    "Rank_2": "loser_rank",
    "Pts_1": "winner_rank_points",
    "Pts_2": "loser_rank_points",
    "Surface": "surface",
    "Date": "tourney_date",
    "Score": "score",
    "Round": "round",
    "Best of": "best_of",
    "Tournament": "tourney_name"
}

# 2. Appliquer le renommage
df_2025_renamed = df_2025.rename(columns=rename_cols)

# 3. Supprimer les colonnes inutiles de Kaggle (non présentes dans Jeff)
columns_to_drop = set(df_2025_renamed.columns) - set(df_jeff.columns)
df_2025_cleaned = df_2025_renamed.drop(columns=columns_to_drop)

# 4. Ajouter les colonnes manquantes de Jeff (avec NaN ou 0 selon le type)
for col in df_jeff.columns:
    if col not in df_2025_cleaned.columns:
        dtype = df_jeff[col].dtype
        default = 0 if np.issubdtype(dtype, np.number) else np.nan
        df_2025_cleaned[col] = default

# 5. Réordonner les colonnes pour matcher exactement Jeff
df_2025_cleaned = df_2025_cleaned[df_jeff.columns]

# Résultat final
print("Harmonisation terminée.")
print(f"Final shape: {df_2025_cleaned.shape}")
df_2025_cleaned.head(3)


Harmonisation terminée.
Final shape: (958, 49)


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
64181,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,0,0,NaN,...,0,0,0,0,0,0,69,776,28,1660
64182,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,0,0,NaN,...,0,0,0,0,0,0,35,1370,75,730
64183,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,0,0,NaN,...,0,0,0,0,0,0,68,778,10,3350


In [40]:
col = ["winner_name", "winner_id"]

df_2025_cleaned[col]


,winner_name,winner_id
64181,Nishioka Y.,0
64182,Jarry N.,0
64183,Vukic A.,0
64184,Thompson J.,0
64185,Lajovic D.,0
...,...,...
65146,Draper J.,0
65147,Diallo G.,0
65148,Ruud C.,0
65149,Musetti L.,0


In [ ]:
# Chargement du fichier des joueurs ATP
players = pd.read_csv("../../Data/players/atp_players.csv")

# Ajout d'une colonne d'initiales pour faciliter le matching
players["initial"] = players["name_first"].str[0].str.upper()
players["name_last"] = players["name_last"].str.strip().str.upper()

# Nettoyage des noms du dataset Kaggle
def extract_last_and_initial(fullname):
    try:
        last, initial = fullname.strip().split(" ")
        return last.upper(), initial.replace(".", "").upper()
    except:
        return None, None

# Appliquer la fonction pour extraire le nom et l'initiale
df_2025_cleaned["winner_last"], df_2025_cleaned["winner_initial"] = zip(*df_2025_cleaned["winner_name"].map(extract_last_and_initial))
df_2025_cleaned["loser_last"],  df_2025_cleaned["loser_initial"]  = zip(*df_2025_cleaned["loser_name"].map(extract_last_and_initial))

# Merge winner
df_2025_cleaned = df_2025_cleaned.merge(
    players[["player_id", "name_last", "initial"]],
    left_on=["winner_last", "winner_initial"],
    right_on=["name_last", "initial"],
    how="left"
).rename(columns={"player_id": "winner_id"}).drop(columns=["name_last", "initial"])

# Merge loser
df_2025_cleaned = df_2025_cleaned.merge(
    players[["player_id", "name_last", "initial"]],
    left_on=["loser_last", "loser_initial"],
    right_on=["name_last", "initial"],
    how="left"
).rename(columns={"player_id": "loser_id"}).drop(columns=["name_last", "initial"])

# Supprimer les colonnes temporaires
df_2025_cleaned.drop(columns=["winner_last", "winner_initial", "loser_last", "loser_initial"], inplace=True)

# Supprimer colonnes dupliquées (pandas bug si noms répétés)
df_2025_cleaned = df_2025_cleaned.loc[:, ~df_2025_cleaned.columns.duplicated()]

# Vérification
missing = df_2025_cleaned[df_2025_cleaned["winner_id"].isna() | df_2025_cleaned["loser_id"].isna()]
print(f"⚠️  Matchs sans ID trouvé : {len(missing)} sur {len(df_2025_cleaned)}")


/tmp/ipykernel_150380/2518441324.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv("../../Data/players/atp_players.csv")
